# Creating Final Aggregated Dataset
## Process used for weeks 1 through 9
#### This is the snippet for Week 9

In [1]:
!pip install google-cloud-bigquery google-cloud-storage

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
project_id = "bigdatabowl2025"
client = bigquery.Client(project=project_id, location='us-central1')

In [5]:
# List of table names and corresponding DataFrame names
tables = {
    "players": "players_df",
    "games" : "games_df",
    "player_play" : "player_play_df",
    "plays" : "plays_df",
    # "tracking_week_1": "tracking_week_1",
    # "tracking_week_2": "tracking_week_2",
    # "tracking_week_3": "tracking_week_3",
    # "tracking_week_4": "tracking_week_4",
    # "tracking_week_5": "tracking_week_5",
    # "tracking_week_6": "tracking_week_6",
    # "tracking_week_7": "tracking_week_7",
    # "tracking_week_8": "tracking_week_8",
    "tracking_week_9": "tracking_week_9"
}

# Dataset details
dataset_id = "bdb_tables"

# Query each table and assign to DataFrame variables
for table_name, df_name in tables.items():
    query = f"SELECT * FROM `{client.project}.{dataset_id}.{table_name}`"
    print(f"Querying table: {table_name}")
    globals()[df_name] = client.query(query).to_dataframe()

    print(f"Loaded {len(globals()[df_name])} rows into DataFrame: {df_name}")

Querying table: players
Loaded 1697 rows into DataFrame: players_df
Querying table: games
Loaded 136 rows into DataFrame: games_df
Querying table: player_play
Loaded 354727 rows into DataFrame: player_play_df
Querying table: plays
Loaded 16124 rows into DataFrame: plays_df
Querying table: tracking_week_9
Loaded 5104643 rows into DataFrame: tracking_week_9


### Importing Data

In [6]:
import numpy as np
import os

pd.set_option('display.max_columns', None)


In [7]:
tracking = tracking_week_9

In [8]:
# Select only necessary columns from each DataFrame for this analysis
plays_df = plays_df[['gameId', 'playId', 'rushLocationType', 'quarter', 'down', 'yardsToGo', 'gameClock',
       'possessionTeam', 'preSnapHomeScore', 'preSnapVisitorScore', 'absoluteYardlineNumber',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'expectedPoints', 'offenseFormation', 'receiverAlignment', 'pff_passCoverage', 'pff_manZone']]
players_df = players_df[['nflId', 'position']]
games_df = games_df[['gameId', 'gameDate', 'gameTimeEastern', 'homeTeamAbbr']]
player_play_df = player_play_df[['gameId', 'playId', 'nflId', 'inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset']]
tracking_df = tracking[['gameId', 'playId', 'nflId', 'displayName', 'x', 'y', 'event']]

In [9]:
# Check for and handle invalid values in nflId
tracking_df['nflId'] = pd.to_numeric(tracking_df['nflId'], errors='coerce')  # Convert non-numeric to NaN
tracking_df['nflId'] = tracking_df['nflId'].fillna(-1).astype('int32')       # Fill NaN with -1 and convert to int32

# Adjust data types for memory efficiency using .loc[] to avoid SettingWithCopyWarning
games_df.loc[:, 'gameId'] = games_df['gameId'].astype('int32')
plays_df.loc[:, 'gameId'] = plays_df['gameId'].astype('int32')
plays_df.loc[:, 'playId'] = plays_df['playId'].astype('int32')
player_play_df.loc[:, 'gameId'] = player_play_df['gameId'].astype('int32')
player_play_df.loc[:, 'playId'] = player_play_df['playId'].astype('int32')
tracking_df.loc[:, 'gameId'] = tracking_df['gameId'].astype('int32')
tracking_df.loc[:, 'playId'] = tracking_df['playId'].astype('int32')
players_df.loc[:, 'nflId'] = players_df['nflId'].astype('int32')
player_play_df.loc[:, 'nflId'] = player_play_df['nflId'].astype('int32')


<ipython-input-9-58ba083fa3be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracking_df['nflId'] = pd.to_numeric(tracking_df['nflId'], errors='coerce')  # Convert non-numeric to NaN
<ipython-input-9-58ba083fa3be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracking_df['nflId'] = tracking_df['nflId'].fillna(-1).astype('int32')       # Fill NaN with -1 and convert to int32


In [10]:
# Join the DataFrames

# Join plays and tracking on 'gameId' and 'playId'
join_all = pd.merge(plays_df, tracking_df, how="inner", on=["gameId", "playId"])

# Join the resulting DataFrame with players on 'nflId'
join_all = join_all.merge(players_df, how="left", on="nflId")

join_all = join_all.merge(player_play_df, how="left", on=["gameId", "playId", "nflId"])

join_all = join_all.merge(games_df, how="left", on='gameId')

# Display information on the joined DataFrame
print("Final joined DataFrame shape:", join_all.shape)
print(join_all.head())

Final joined DataFrame shape: (5104643, 30)
       gameId  playId rushLocationType  quarter  down  yardsToGo gameClock  \
0  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
1  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
2  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
3  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
4  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   

  possessionTeam  preSnapHomeScore  preSnapVisitorScore  \
0            PHI                 7                    7   
1            PHI                 7                    7   
2            PHI                 7                    7   
3            PHI                 7                    7   
4            PHI                 7                    7   

   absoluteYardlineNumber  preSnapHomeTeamWinProbability  \
0                      44                       0.149478   
1           

In [11]:
pred_df = join_all
print(pred_df['playId'].nunique())
print(pred_df['gameId'].nunique())
print(len(pred_df))

1276
13
5104643


In [12]:
print(pred_df['event'].value_counts())

event
line_set                     31441
ball_snap                    31395
huddle_break_offense         25185
first_contact                20608
tackle                       19642
pass_forward                 16583
pass_arrived                 12535
handoff                      11316
pass_outcome_caught           9867
man_in_motion                 9821
play_action                   5819
pass_outcome_incomplete       5520
shift                         5474
out_of_bounds                 3358
autoevent_passinterrupted     2737
run                           2277
qb_sack                       1035
touchdown                      782
play_submit                    713
dropped_pass                   644
pass_outcome_touchdown         437
pass_tipped                    345
lateral                        322
fumble                         299
qb_slide                       299
pass_outcome_interception      276
fumble_offense_recovered       230
qb_kneel                       230
fumble_defense

In [13]:
# Fill 'position' with 'FBall' when 'displayName_x' is 'football'
pred_df.loc[pred_df['displayName'] == 'football', 'position'] = 'FBall'

# Filter pred_df for specific events and positions
filtered_pred_df = pred_df[(pred_df['event'] == 'ball_snap')]
filtered_pred_df = filtered_pred_df[filtered_pred_df['position'].isin(['TE', 'WR', 'FB', 'RB', 'QB', 'FBall'])]

# Create new feature identifying if the home team has possession
filtered_pred_df['homeTeamPossession'] = (filtered_pred_df['possessionTeam'] == filtered_pred_df['homeTeamAbbr']).astype(int)

# Drop uneeded columns
filtered_pred_df = filtered_pred_df.drop(columns=['displayName', 'homeTeamAbbr'])

# Display the first few rows of the filtered DataFrame
print(filtered_pred_df.head())

          gameId  playId rushLocationType  quarter  down  yardsToGo gameClock  \
2172  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
2298  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
2466  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
3652  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
3682  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   

     possessionTeam  preSnapHomeScore  preSnapVisitorScore  \
2172            PHI                 7                    7   
2298            PHI                 7                    7   
2466            PHI                 7                    7   
3652            PHI                 7                    7   
3682            PHI                 7                    7   

      absoluteYardlineNumber  preSnapHomeTeamWinProbability  \
2172                      44                       0.149478   
2298          

In [14]:
print(filtered_pred_df[['inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset']].value_counts())

inMotionAtBallSnap  shiftSinceLineset  motionSinceLineset
FALSE               FALSE              FALSE                 3473
NA                  FALSE              NA                    3020
FALSE               FALSE              NA                     736
NA                  TRUE               NA                     239
TRUE                FALSE              NA                     194
FALSE               FALSE              TRUE                   193
TRUE                FALSE              TRUE                   191
FALSE               TRUE               TRUE                    37
                                       NA                      31
TRUE                TRUE               NA                      25
NA                  NA                 NA                      11
Name: count, dtype: int64


In [15]:
# Replace 'FALSE', 'TRUE', and 'NA' strings with appropriate values
filtered_pred_df['inMotionAtBallSnap'] = filtered_pred_df['inMotionAtBallSnap'].replace({'FALSE': False, 'TRUE': True, 'NA': 0}).fillna(0).astype(int)
filtered_pred_df['shiftSinceLineset'] = filtered_pred_df['shiftSinceLineset'].replace({'FALSE': False, 'TRUE': True, 'NA': 0}).fillna(0).astype(int)
filtered_pred_df['motionSinceLineset'] = filtered_pred_df['motionSinceLineset'].replace({'FALSE': False, 'TRUE': True, 'NA': 0}).fillna(0).astype(int)

# Recreate encoded columns for each position
positions = ['RB', 'WR', 'FB', 'TE', 'QB']
for pos in positions:
    filtered_pred_df[f'{pos}_snap_motion'] = np.where(filtered_pred_df['position'] == pos, filtered_pred_df['inMotionAtBallSnap'], 0)
    filtered_pred_df[f'{pos}_motion'] = np.where(filtered_pred_df['position'] == pos, filtered_pred_df['motionSinceLineset'], 0)
    filtered_pred_df[f'{pos}_shift'] = np.where(filtered_pred_df['position'] == pos, filtered_pred_df['shiftSinceLineset'], 0)

# Check the resulting DataFrame
print(filtered_pred_df.head())


          gameId  playId rushLocationType  quarter  down  yardsToGo gameClock  \
2172  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
2298  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
2466  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
3652  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   
3682  2022110300    1182      INSIDE_LEFT        2     1         10     09:28   

     possessionTeam  preSnapHomeScore  preSnapVisitorScore  \
2172            PHI                 7                    7   
2298            PHI                 7                    7   
2466            PHI                 7                    7   
3652            PHI                 7                    7   
3682            PHI                 7                    7   

      absoluteYardlineNumber  preSnapHomeTeamWinProbability  \
2172                      44                       0.149478   
2298          

In [16]:
# List of columns to check
columns_to_check = [
    'inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset', 'RB_snap_motion',
    'WR_snap_motion', 'FB_snap_motion', 'TE_snap_motion', 'QB_snap_motion',
    'RB_motion', 'WR_motion', 'FB_motion', 'TE_motion', 'QB_motion',
    'RB_shift', 'WR_shift', 'FB_shift', 'TE_shift', 'QB_shift'
]

# Display value counts for each column
for column in columns_to_check:
    print(f"Value counts for {column}:")
    print(filtered_pred_df[column].value_counts())
    print("\n")

Value counts for inMotionAtBallSnap:
inMotionAtBallSnap
0    9105
1     410
Name: count, dtype: int64


Value counts for shiftSinceLineset:
shiftSinceLineset
0    9183
1     332
Name: count, dtype: int64


Value counts for motionSinceLineset:
motionSinceLineset
0    9094
1     421
Name: count, dtype: int64


Value counts for RB_snap_motion:
RB_snap_motion
0    9475
1      40
Name: count, dtype: int64


Value counts for WR_snap_motion:
WR_snap_motion
0    9255
1     260
Name: count, dtype: int64


Value counts for FB_snap_motion:
FB_snap_motion
0    9503
1      12
Name: count, dtype: int64


Value counts for TE_snap_motion:
TE_snap_motion
0    9418
1      97
Name: count, dtype: int64


Value counts for QB_snap_motion:
QB_snap_motion
0    9514
1       1
Name: count, dtype: int64


Value counts for RB_motion:
RB_motion
0    9421
1      94
Name: count, dtype: int64


Value counts for WR_motion:
WR_motion
0    9312
1     203
Name: count, dtype: int64


Value counts for FB_motion:
FB_motion


In [17]:
# Seeing how many records with given position
for pos in ['RB', 'WR', 'FB', 'TE', 'QB']:
    print(f"Occurrences for {pos}:")
    print(filtered_pred_df[filtered_pred_df['position'] == pos].shape[0])


Occurrences for RB:
1392
Occurrences for WR:
3483
Occurrences for FB:
113
Occurrences for TE:
1786
Occurrences for QB:
1376


In [18]:
# Filter the DataFrame for the 'ball_snap' event
filtered_ball_snap_df = filtered_pred_df[filtered_pred_df['event'] == 'ball_snap']

# Initialize new columns with NaN
filtered_pred_df['qb_depth'] = None
filtered_pred_df['rb_depth'] = None
filtered_pred_df['fb_depth'] = None

# Calculate qb_depth, rb_depth, fb_depth for each play
for (gameId, playId), group in filtered_ball_snap_df.groupby(['gameId', 'playId']):
    # Get positions for the current group
    fball_x = group.loc[group['position'] == 'FBall', 'x']
    qb_x = group.loc[group['position'] == 'QB', 'x']
    rb_x = group.loc[group['position'] == 'RB', 'x']
    fb_x = group.loc[group['position'] == 'FB', 'x']

    # Ensure positions exist and calculate depths
    if not fball_x.empty:
        fball_x = fball_x.values[0]
        if not qb_x.empty:
            qb_depth = abs(fball_x - qb_x.values[0])  # Absolute difference
            filtered_pred_df.loc[
                (filtered_pred_df['gameId'] == gameId) &
                (filtered_pred_df['playId'] == playId), 'qb_depth'] = qb_depth
        if not rb_x.empty:
            rb_depth = abs(fball_x - rb_x.values[0])  # Absolute difference
            filtered_pred_df.loc[
                (filtered_pred_df['gameId'] == gameId) &
                (filtered_pred_df['playId'] == playId), 'rb_depth'] = rb_depth
        if not fb_x.empty:
            fb_depth = abs(fball_x - fb_x.values[0])  # Absolute difference
            filtered_pred_df.loc[
                (filtered_pred_df['gameId'] == gameId) &
                (filtered_pred_df['playId'] == playId), 'fb_depth'] = fb_depth

# Verify the result
print(filtered_pred_df[['gameId', 'playId', 'qb_depth', 'rb_depth', 'fb_depth']].head(100))


           gameId  playId  qb_depth  rb_depth fb_depth
2172   2022110300    1182  1.500001  6.960001     None
2298   2022110300    1182  1.500001  6.960001     None
2466   2022110300    1182  1.500001  6.960001     None
3652   2022110300    1182  1.500001  6.960001     None
3682   2022110300    1182  1.500001  6.960001     None
...           ...     ...       ...       ...      ...
53964  2022110300    2035      1.66       7.3     None
54873  2022110300    2035      1.66       7.3     None
54987  2022110300    2035      1.66       7.3     None
58011  2022110300    1091  1.340001  6.960001     None
58665  2022110300    1091  1.340001  6.960001     None

[100 rows x 5 columns]


In [19]:
# Define aggregation logic
aggregated_df = (
    filtered_pred_df.groupby(['gameId', 'playId'], as_index=False).agg({
        'rushLocationType': lambda x: 1 if any(val not in ['UNKNOWN', 'NA'] for val in x) else 0,
        'x': lambda x: x[filtered_pred_df['position'] == 'FB'].mean() - x[filtered_pred_df['position'] == 'QB'].mean(),  # player depth
        'qb_depth': 'max',
        'rb_depth': 'max',
        'fb_depth': 'max',
        'absoluteYardlineNumber': 'first',
        'preSnapHomeTeamWinProbability': 'first',
        'preSnapVisitorTeamWinProbability': 'first',
        'expectedPoints': 'first',
        'offenseFormation': 'first',
        'receiverAlignment': 'first',
        'pff_passCoverage': 'first',
        'pff_manZone': 'first',
        'homeTeamPossession': 'first',
        'quarter': 'first',
        'down': 'first',
        'yardsToGo': 'first',
        'gameClock': 'first',
        'possessionTeam': 'first',
        'preSnapHomeScore': 'first',
        'preSnapVisitorScore': 'first',
        'RB_snap_motion': 'max',
        'WR_snap_motion': 'max',
        'FB_snap_motion': 'max',
        'TE_snap_motion': 'max',
        'QB_snap_motion': 'max',
        'RB_motion': 'max',
        'WR_motion': 'max',
        'FB_motion': 'max',
        'TE_motion': 'max',
        'QB_motion': 'max',
        'RB_shift': 'max',
        'WR_shift': 'max',
        'FB_shift': 'max',
        'TE_shift': 'max',
        'QB_shift': 'max'
        })
)

# Replace NaN values in rb_depth and fb_depth with 0
aggregated_df['rb_depth'] = aggregated_df['rb_depth'].fillna(0)
aggregated_df['fb_depth'] = aggregated_df['fb_depth'].fillna(0)

# Count the number of unique combinations of gameId and playId
unique_combinations = aggregated_df.groupby(['gameId', 'playId']).ngroups
print(f"Number of unique combinations of gameId and playId: {unique_combinations}")

print(aggregated_df.shape)
print(aggregated_df.head())

Number of unique combinations of gameId and playId: 1365
(1365, 38)
       gameId  playId  rushLocationType     x  qb_depth  rb_depth  fb_depth  \
0  2022110300      55                 1   NaN      4.66  4.990000  0.000000   
1  2022110300      76                 0   NaN  1.409999  7.699999  0.000000   
2  2022110300     100                 1   NaN       5.1  5.300000  0.000000   
3  2022110300     121                 1 -3.45  1.440001  7.740001  4.890001   
4  2022110300     142                 1   NaN      1.36  7.450000  0.000000   

   absoluteYardlineNumber  preSnapHomeTeamWinProbability  \
0                      35                       0.148526   
1                      46                       0.160490   
2                      54                       0.171295   
3                      59                       0.181043   
4                      62                       0.180569   

   preSnapVisitorTeamWinProbability  expectedPoints offenseFormation  \
0                       

<ipython-input-19-3cf8004c5848>:44: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aggregated_df['rb_depth'] = aggregated_df['rb_depth'].fillna(0)
<ipython-input-19-3cf8004c5848>:45: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aggregated_df['fb_depth'] = aggregated_df['fb_depth'].fillna(0)


In [20]:
# Drop unnecessary columns
aggregated_df = aggregated_df.drop(columns=['expectedPoints', 'x', 'QB_motion', 'QB_shift', 'QB_snap_motion'], errors='ignore')

# Rename 'rushLocationType' to 'target_rush'
aggregated_df = aggregated_df.rename(columns={'rushLocationType': 'target_rush'})

# Verify the result
print(aggregated_df.head())


       gameId  playId  target_rush  qb_depth  rb_depth  fb_depth  \
0  2022110300      55            1      4.66  4.990000  0.000000   
1  2022110300      76            0  1.409999  7.699999  0.000000   
2  2022110300     100            1       5.1  5.300000  0.000000   
3  2022110300     121            1  1.440001  7.740001  4.890001   
4  2022110300     142            1      1.36  7.450000  0.000000   

   absoluteYardlineNumber  preSnapHomeTeamWinProbability  \
0                      35                       0.148526   
1                      46                       0.160490   
2                      54                       0.171295   
3                      59                       0.181043   
4                      62                       0.180569   

   preSnapVisitorTeamWinProbability offenseFormation receiverAlignment  \
0                          0.851474          SHOTGUN               3x1   
1                          0.839510       SINGLEBACK               3x1   
2       

In [21]:
# Verifying no null values exist
print(aggregated_df.isnull().sum())

gameId                              0
playId                              0
target_rush                         0
qb_depth                            0
rb_depth                            0
fb_depth                            0
absoluteYardlineNumber              0
preSnapHomeTeamWinProbability       0
preSnapVisitorTeamWinProbability    0
offenseFormation                    0
receiverAlignment                   0
pff_passCoverage                    0
pff_manZone                         0
homeTeamPossession                  0
quarter                             0
down                                0
yardsToGo                           0
gameClock                           0
possessionTeam                      0
preSnapHomeScore                    0
preSnapVisitorScore                 0
RB_snap_motion                      0
WR_snap_motion                      0
FB_snap_motion                      0
TE_snap_motion                      0
RB_motion                           0
WR_motion   

In [22]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq

dataset_table = f"{dataset_id}.aggregated_week9_df"

# Write the DataFrame to BigQuery
pandas_gbq.to_gbq(
    aggregated_df,
    dataset_table,
    project_id=project_id,
    if_exists='replace'  # Options: 'fail', 'replace', 'append'
)

print(f"DataFrame saved to BigQuery table {dataset_table}")


100%|██████████| 1/1 [00:00<00:00, 1060.24it/s]

DataFrame saved to BigQuery table bdb_tables.aggregated_week9_df
